In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates
from format_the_data import format_mi_band_data
from feature_engineering import get_wake_up_info_miband, get_wake_up_info_applewatch, get_heartrate_data_for_interval, calculate_test_statistics_heartrate
from load_the_data import process_fitness_data
import json
from pprint import pprint
pd.options.mode.chained_assignment = None

In [ ]:
### set os dir to data, same director but in folder data
os.chdir('data')
person1 = pd.read_csv('20231030_8210796956_MiFitness_hlth_center_fitness_data.csv')
person2 = pd.read_csv('20231030_8211531339_MiFitness_hlth_center_fitness_data.csv')
person3 = pd.read_csv('20231031_8210564343_MiFitness_hlth_center_fitness_data.csv')
person4 = pd.read_csv('20231110_8210586841_MiFitness_hlth_center_fitness_data.csv')

person1['Person ID'] = 1
person2['Person ID'] = 2
person3['Person ID'] = 3
person4['Person ID'] = 4

apple_sleep_data = pd.read_csv('sleep_apple.csv')
apple_heart_rate_data = pd.read_csv('heart_rate_apple.csv')

master_frame = pd.concat([person1, person2, person3, person4], ignore_index=True)

behaviour_tracking_data = pd.read_csv('Behavioural data app.csv')

In [ ]:
aggregated_df = behaviour_tracking_data.copy()

In [4]:
unique_keys = ['pai',
               'valid_stand', 
               'calories',
               'steps',
               'heart_rate',
               'intensity',
               'dynamic',
               'single_heart_rate',
               'single_spo2',
               'training_load',
               'single_stress',
               'stress',
               'watch_night_sleep',
               'resting_heart_rate',
               'watch_daytime_sleep',
               'weight']
key_dataframes = format_mi_band_data(unique_keys, master_frame)


# separate dataframes:
pai_df = key_dataframes['pai']
valid_stand_df = key_dataframes['valid_stand']
calories_df = key_dataframes['calories']
steps_df = key_dataframes['steps']
heart_rate_df = key_dataframes['heart_rate']
intensity_df = key_dataframes['intensity']
dynamic_df = key_dataframes['dynamic']
single_heart_rate_df = key_dataframes['single_heart_rate']
single_spo2_df = key_dataframes['single_spo2']
training_load_df = key_dataframes['training_load']
single_stress_df = key_dataframes['single_stress']
stress_df = key_dataframes['stress']
watch_night_sleep_df = key_dataframes['watch_night_sleep']
resting_heart_rate_df = key_dataframes['resting_heart_rate']
watch_daytime_sleep_df = key_dataframes['watch_daytime_sleep']
weight_df = key_dataframes['weight']

# Intuition

I think we need to include control variables such as the number of hours sleep and the quality of sleep as this significantly influences your mental state after awaking.

I guess we can check whether someone has slept after their alarm right?

We must include variable for smart alarm or not. Can we find this in the settings from the mi band?

We are going to add features from our watch data to the behavioural data.

Features for our analysis:
- Average of the first 20 minutes after waking
- Whether the smart alarm actually woke us up in light sleep (not in deep or REM)

Hence we need:
- Waking time
- Heartrate data
- State when awaking

In [5]:
aggregated_df['Person ID'].value_counts()

Person ID
1    28
2    27
5    26
4    23
3    15
Name: count, dtype: int64

## Aggregate sleepdata and add to aggregated_df (with behavioural data)

In [6]:
aggregated_df['Date'] = pd.to_datetime(aggregated_df['Date created']).dt.date

# for apple watch
aggregated_df_person_5 = aggregated_df[aggregated_df['Person ID'] == 5]
aggregated_df_person_5 = get_wake_up_info_applewatch(apple_sleep_data, aggregated_df_person_5)

# for mi band
aggregated_df_rest = aggregated_df[aggregated_df['Person ID'] != 5]
aggregated_df_rest = get_wake_up_info_miband(watch_night_sleep_df, aggregated_df_rest)

# aggregate
aggregated_df = pd.concat([aggregated_df_person_5, aggregated_df_rest], ignore_index=True)


In [7]:
print(len(aggregated_df))
# filter out the rows where the time_of_awakening is NaN
aggregated_df = aggregated_df[~aggregated_df['time_of_awakening'].isna()].reset_index(drop=True)
print(len(aggregated_df))

119
111


Here 8 observations could not be linked due to missing sleepdata caused by Xiaomi export fail

## Lets aggregate the heart rate for the devices

In [12]:
# Only keep the relevant columns of heart rate data
heart_rate_df = heart_rate_df[['Person ID', 'Time', 'bpm']]

# Prepare apple_heart_rate_data for merging
apple_data_prepared = apple_heart_rate_data[['creationDate', 'value']].copy()
apple_data_prepared.rename(columns={'value': 'bpm'}, inplace=True)
apple_data_prepared['Person ID'] = 5

# Convert 'creationDate' to Unix epoch time (seconds since epoch)
apple_data_prepared['creationDate'] = pd.to_datetime(apple_data_prepared['creationDate'])
apple_data_prepared['Time'] = apple_data_prepared['creationDate'].astype('int64') // 10**9

# Drop the original 'creationDate' column
apple_data_prepared.drop('creationDate', axis=1, inplace=True)

# Use concat instead of append
heart_rate_df = pd.concat([heart_rate_df, apple_data_prepared], ignore_index=True)

## Next we get the insights and add to aggregated_df

In [13]:
time_interval = 60  # Assuming a 60 minute interval

# Add new columns for the statistics in behaviour_tracking_data
aggregated_df['Number of Measurements'] = pd.NA
aggregated_df['Average Heart Rate'] = pd.NA
aggregated_df['Average Lowest Three obs'] = pd.NA
aggregated_df['Average First Ten min'] = pd.NA
aggregated_df['Average First Thirty min'] = pd.NA

for index, row in aggregated_df.iterrows():
    # Get heart rate data for the interval
    heart_rate_data = get_heartrate_data_for_interval(heart_rate_df, row['Person ID'], row['Date'], row['time_of_awakening'], time_interval)
    
    # Calculate the test statistics
    num_measurements, avg_hr, avg_lowest_three, avg_first_ten, avg_first_thirty = calculate_test_statistics_heartrate(heart_rate_data)

    # Update the aggregated_df DataFrame with the new statistics
    aggregated_df.at[index, 'Number of Measurements'] = num_measurements
    aggregated_df.at[index, 'Average Heart Rate'] = avg_hr
    aggregated_df.at[index, 'Average Lowest Three obs'] = avg_lowest_three
    aggregated_df.at[index, 'Average First Ten min'] = avg_first_ten
    aggregated_df.at[index, 'Average First Thirty min'] = avg_first_thirty

## Cleaning out outliers due to sport activity or too little measurements

In [25]:
# Count outliers below 6 and above 100 for each 'Person ID'
outliers_below_6 = aggregated_df[aggregated_df['Number of Measurements'] < 6].groupby('Person ID').size()
outliers_above_100 = aggregated_df[aggregated_df['Number of Measurements'] > 100].groupby('Person ID').size()

# Filter out the outliers
filtered_df = aggregated_df[(aggregated_df['Number of Measurements'] >= 6) & (aggregated_df['Number of Measurements'] <= 100)]

# Check and report the counts of outliers removed
if not outliers_below_6.empty or not outliers_above_100.empty:
    if not outliers_below_6.empty:
        print("Outliers removed for each 'Person ID' (below 6):")
        print(outliers_below_6)
    if not outliers_above_100.empty:
        print("\nOutliers removed for each 'Person ID' (above 100):")
        print(outliers_above_100)
else:
    print("No outliers in your data!")



Outliers removed for each 'Person ID' (above 100):
Person ID
5    5
dtype: int64


In [26]:
# replace all False values with 0 and True values with 1 in aggregated_df
aggregated_df = aggregated_df.replace(False, 0)
aggregated_df = aggregated_df.replace(True, 1)


In [27]:
# Save the aggregated_df DataFrame to a csv file
aggregated_df.to_csv('aggregated_df.csv', index=False)